## Imports and settings

In [2]:
import experiments as E
import xml.etree.ElementTree as ET   # needed for Valgrind outputs
import os
import pandas as pd
import numpy as np
import re
import toml           # needed for Binsec/Rel2 outputs
import csv            # needed for Binsec/Rel1 outputs
from datetime import timedelta, time
from typing import List

pd.set_option('styler.latex.hrules', True)
pd.set_option('styler.format.precision', 2)


## Common functions and values

In [3]:
def load_text_output(path: str) -> List[str]:
    output = ""
    try: 
        with open(path, "r") as f:
            output = f.readlines()
    except OSError:
        return []
        
    return output

def init_dict(keys, value):
    return {k:v for (k,v) in zip(keys, [value for _ in range(len(keys))])}

BENCH_RES_DIR = "benchmarks-results"
VULN_RES_DIR = "vulns-results"
# experiment names in the order defined in experiments.py
binary_names = [b.name for benchmarks in E.all_bench for b in benchmarks]
vulns_names = [v.name for vulns in E.all_vuln for v in vulns]

print(binary_names)
print(vulns_names)


['aes_big_cbc-GCC9-O2', 'aes_ct_cbc-GCC9-O2', 'aes_big_gcm-GCC9-O2', 'aes_ct_gcm-GCC9-O2', 'aes_crypt_cbc-GCC9-O2', 'gcm_starts-GCC9-O2', 'evp_aes_cbc-GCC9-O2', 'evp_aes_gcm-GCC9-O2', 'aes_cbc_encrypt-GCC9-O2', 'vpaes_cbc_encrypt-GCC9-O2', 'poly1305_ctmul_chacha20_ct-GCC9-O2', 'poly1305_ctmul_chacha20_sse2-GCC9-O2', 'mbedtls_chachapoly-GCC9-O2', 'evp_chacha20_poly1305-GCC9-O2', 'chacha20_ctr32-GCC9-O2', 'poly1305_update-GCC9-O2', 'rsa_i31_oaep_decrypt-GCC9-O2', 'rsa_pkcs1_v15_decrypt-GCC9-O2', 'rsa_oaep_decrypt-GCC9-O2', 'RSA_private_decrypt_pkcs1_v15-GCC9-O2', 'RSA_private_decrypt_oaep-GCC9-O2', 'ecdsa_i31_sign_p256-GCC9-O2', 'ecdsa_sign_det_ext_p256-GCC9-O2', 'ECDSA_do_sign_p256-GCC9-O2', 'ED25519_sign-GCC9-O2']
['ECDSA_do_sign-GCC9-O2', 'ec_wnaf_mul-GCC9-O2', 'mbedtls_rsa_complete-GCC9-O2', 'mbedtls_mpi_gcd-GCC9-O2', 'mbedtls_mpi_inv_mod-GCC9-O2', 'RSA_key_generate_ex-GCC9-O2', 'bn_gcd-GCC9-O2', 'bn_mod_exp_mont-GCC9-O2', 'bn_mod_inverse-GCC9-O2', 'gcry_pk_decrypt-GCC9-O2', 'gcry_mp

## ctgrind

ctgrind's outputs are in the form of one XML file per binary analyzed, listing vulnerabilities, their context and their origin *if* Valgrind is able to determine it. From this list of vulnerabilities we are interested in conditions and memory accesses computed using uninitialized memory, which comes from our call to `VALGRIND_MAKE_MEM_UNDEFINED` (so-called "client request").

For vulnerabilities whose origin *cannot* be traced by Valgrind, we can either choose to keep them (potentially adding false positives) or ignore them (potentially missing real vulnerabilities). By default **we keep these vulnerabilities**.

Additionnally, Valgrind reports a vulnerability if its instruction pointer *and* calling context are different. Thus a single leakage point can be reported multiple times. By defaut, **we filter out these duplicates**.

In [8]:

MEMCHECK_COND = "UninitCondition"
MEMCHECK_VALUE = "UninitValue"
CLIENT_ORIGIN = "Uninitialised value was created by a client request"

def load_ctgrind_output(path: str):
    output = ""
    with open(path, "r") as f:
        output = f.readlines()
    
    root = ET.fromstringlist(output)
    return root

def ctgrind_vuln(root: ET.Element, keep_unknown_origins=True, filter_duplicates=True):
    elem_errors = root.findall("error")
    vulns = []

    for elem in elem_errors:
        kind = elem.findtext("kind")
        # only keep uninitialized memory errors
        if kind == MEMCHECK_COND or kind == MEMCHECK_VALUE:
            origin = elem.findtext("auxwhat")
            # keep errors coming from client requests
            # or of unknown origins *if* we keep them
            if origin == CLIENT_ORIGIN or (origin == None and keep_unknown_origins):
                error_ip = elem.find("stack").find("frame").findtext("ip")[2:]
                error_ip = hex(int(error_ip,16))
                if filter_duplicates:
                    if error_ip not in vulns: vulns.append(error_ip)
                else:
                    vulns.append(error_ip)
    return vulns


In [29]:

RES_DIR = f"{BENCH_RES_DIR}/ctgrind"
ctgrind_vulns = init_dict(binary_names, "NaN")
ctgrind_secures = init_dict(binary_names, None)
ctgrind_times = init_dict(binary_names, "NaT")
stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    root = load_ctgrind_output(f"{RES_DIR}/{name}-ctgrind.txt")
    ctgrind_vulns[name] = len(ctgrind_vuln(root, True, True))
    ctgrind_times[name] = row["ctgrind_time"]
    ctgrind_secures[name] = None if ctgrind_vulns[name] == 0 else False
    
    print(f"{name}: {ctgrind_secures[name]}, {ctgrind_vulns[name]} in {ctgrind_times[name]}")

aes_big_cbc-GCC9-O2: False, 36 in 0.161883592605591
aes_ct_cbc-GCC9-O2: None, 0 in 0.168594598770142
aes_big_gcm-GCC9-O2: False, 36 in 0.17594051361084
aes_ct_gcm-GCC9-O2: None, 0 in 0.172316789627075
aes_crypt_cbc-GCC9-O2: False, 68 in 0.188573837280273
gcm_starts-GCC9-O2: False, 76 in 0.190616130828857
evp_aes_cbc-GCC9-O2: None, 0 in 0.664029359817505
evp_aes_gcm-GCC9-O2: False, 70 in 0.712273597717285
aes_cbc_encrypt-GCC9-O2: False, 36 in 0.178664445877075
vpaes_cbc_encrypt-GCC9-O2: None, 0 in 0.18757176399231
poly1305_ctmul_chacha20_ct-GCC9-O2: None, 0 in 0.164394378662109
poly1305_ctmul_chacha20_sse2-GCC9-O2: None, 0 in 0.153951168060303
mbedtls_chachapoly-GCC9-O2: None, 0 in 0.180468559265137
evp_chacha20_poly1305-GCC9-O2: None, 0 in 0.672555923461914
chacha20_ctr32-GCC9-O2: None, 0 in 0.146229028701782
poly1305_update-GCC9-O2: None, 0 in 0.165188074111939
rsa_i31_oaep_decrypt-GCC9-O2: False, 87 in 0.570716142654419
rsa_pkcs1_v15_decrypt-GCC9-O2: False, 39 in 0.962103128433228
rs

In [9]:
RES_DIR = "vulns-results/ctgrind"
ctgrind_v_vulns = init_dict(vulns_names, "NaN")
ctgrind_v_times = init_dict(vulns_names, "NaT")
ctgrind_v_secures = init_dict(binary_names, None)
stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    root = load_ctgrind_output(f"{RES_DIR}/{name}-ctgrind.txt")
    ctgrind_v_vulns[name] = len(ctgrind_vuln(root))
    ctgrind_v_times[name] = row["ctgrind_time"]
    ctgrind_v_secures[name] = None if ctgrind_v_vulns[name] == 0 else False

    
    print(f"{name}: {ctgrind_v_secures[name]},{ctgrind_v_vulns[name]} in {ctgrind_v_times[name]}")

ECDSA_do_sign-GCC9-O2: None,0 in 0.7452511787414551
ec_wnaf_mul-GCC9-O2: False,222 in 0.5360617637634277
mbedtls_rsa_complete-GCC9-O2: False,31 in 0.40250325202941895
mbedtls_mpi_gcd-GCC9-O2: False,14 in 0.20527863502502441
mbedtls_mpi_inv_mod-GCC9-O2: False,19 in 0.24109482765197754
RSA_key_generate_ex-GCC9-O2: None,0 in 6.362996339797974
bn_gcd-GCC9-O2: False,53 in 0.18973445892333984
bn_mod_exp_mont-GCC9-O2: False,6 in 0.24148178100585938
bn_mod_inverse-GCC9-O2: False,115 in 0.2078719139099121
gcry_pk_decrypt-GCC9-O2: False,359 in 0.9973058700561523
gcry_mpi_mod-GCC9-O2: False,9 in 0.22417569160461426


## Abacus

Abacus' outputs are in the form of simple plaintext files, listing the number of vulnerabilities and their addresses. The total running time of the analysis, as measured in our python script, is contained in `benchmark_stats.csv`

In [10]:

def abacus_vuln(output: List[str]):
    preamble_end = output.index("DETAILS:\n")
    output = [line for line in output[0:preamble_end] if line.startswith("Address:")]
    return [hex(int(line.split(" ")[1],16)) for line in output]


In [12]:
RES_DIR = f"{BENCH_RES_DIR}/abacus"
abacus_vulns = init_dict(binary_names, "NaN")
abacus_secures = init_dict(binary_names, None)
abacus_times = init_dict(binary_names, "NaT")

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    # check if Abacus crashed or not
    if row["abacus_status"] != "0":
        abacus_times[name] = float(row["pin_time"]) + float(row["abacus_time"])
        abacus_vulns[name] = "NaN"
        abacus_secures[name] = None
    else:
        stdout = load_text_output(f"{RES_DIR}/{name}-abacus.txt")    # load Abacus output
        abacus_times[name] = float(row["pin_time"]) + float(row["abacus_time"])
        abacus_vulns[name] = len(abacus_vuln(stdout))
        abacus_secures[name] = None if abacus_vulns[name] == 0 else False

    
    print(f"{name}: {abacus_secures[name]}, {abacus_vulns[name]} in {abacus_times[name]}")

aes_big_cbc-GCC9-O2: False, 36 in 3.6499187946319602
aes_ct_cbc-GCC9-O2: None, 0 in 10.68725156784057
aes_big_gcm-GCC9-O2: False, 36 in 6.22959208488465
aes_ct_gcm-GCC9-O2: None, 0 in 14.79111599922185
aes_crypt_cbc-GCC9-O2: False, 68 in 5.7278327941894505
gcm_starts-GCC9-O2: False, 84 in 261.7630071640018
evp_aes_cbc-GCC9-O2: None, 0 in 103.58683633804321
evp_aes_gcm-GCC9-O2: None, 0 in 104.2670300006867
aes_cbc_encrypt-GCC9-O2: False, 36 in 3.02420878410339
vpaes_cbc_encrypt-GCC9-O2: None, 0 in 1.014586210250855
poly1305_ctmul_chacha20_ct-GCC9-O2: None, 0 in 1.2989044189453152
poly1305_ctmul_chacha20_sse2-GCC9-O2: None, 0 in 1.1655120849609388
mbedtls_chachapoly-GCC9-O2: None, 0 in 5.01686954498291
evp_chacha20_poly1305-GCC9-O2: None, NaN in 100.1940631866455
chacha20_ctr32-GCC9-O2: None, 0 in 3.77366662025451
poly1305_update-GCC9-O2: None, NaN in 1.189515829086307
rsa_i31_oaep_decrypt-GCC9-O2: None, NaN in 356.40908241271944
rsa_pkcs1_v15_decrypt-GCC9-O2: None, NaN in 2193.198983192

In [11]:
RES_DIR = "vulns-results/abacus"
abacus_v_vulns = init_dict(vulns_names, "NaN")
abacus_v_times = init_dict(vulns_names, "NaT")
abacus_v_secures = init_dict(binary_names, None)

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    if row["abacus_status"] != "0":
        abacus_v_times[name] = float(row["pin_time"]) + float(row["abacus_time"])
        abacus_v_vulns[name] = "NaN"
        abacus_v_secures[name] = None
    else:
        stdout = load_text_output(f"{RES_DIR}/{name}-abacus.txt")
        abacus_v_times[name] = float(row["pin_time"]) + float(row["abacus_time"])
        abacus_v_vulns[name] = len(abacus_vuln(stdout))
        abacus_v_secures[name] = None if abacus_v_vulns[name] == 0 else False
    
    print(f"{name}: {abacus_v_secures[name]},{abacus_v_vulns[name]} in {abacus_v_times[name]}")

ECDSA_do_sign-GCC9-O2: None,NaN in 5.686283349990845
ec_wnaf_mul-GCC9-O2: None,NaN in 3600.355891942978
mbedtls_rsa_complete-GCC9-O2: None,0 in 490.00531554222107
mbedtls_mpi_gcd-GCC9-O2: None,0 in 37.7402069568634
mbedtls_mpi_inv_mod-GCC9-O2: None,0 in 242.0957329273224
RSA_key_generate_ex-GCC9-O2: None,NaN in 8.65596079826355
bn_gcd-GCC9-O2: None,NaN in 3600.1091063022614
bn_mod_exp_mont-GCC9-O2: False,4 in 110.73410868644714
bn_mod_inverse-GCC9-O2: None,NaN in 3600.0259590148926
gcry_pk_decrypt-GCC9-O2: None,0 in 386.762122631073
gcry_mpi_mod-GCC9-O2: None,0 in 2.3508689403533936


## dudect

dudect outputs the result of its t-test score for each batch of measurements, and continues until either the score is too high ("probably not constant-time") or it is timed out by our python script. The total running time of the analysis, as measured in our python script, is appended at the end of the file. 

In [13]:

def dudect_secure(output: List[str]):
    if output == []:
        # dudect couldn't produce an output before timeout
        return None
    
    result_line = output[-1].split(" ")
    if len(result_line) > 3 and result_line[-3] == "not":
        return False
    else:
        return None

def dudect_time(output: List[str]):
    # in any case, the analysis running time should be the only thing on the last line
    return timedelta(seconds=float(output[-1]))


In [12]:

RES_DIR = f"{BENCH_RES_DIR}/dudect"
dudect_secures = init_dict(binary_names, None)
dudect_times = init_dict(binary_names, "Nat")

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    stdout = load_text_output(f"{RES_DIR}/{name}-dudect.txt")
    
    dudect_secures[name] = dudect_secure(stdout)
    dudect_times[name] = row["dudect_time"]
    
    print(f"{name}: {dudect_secures[name]} in {dudect_times[name]}")


NameError: name 'dudect_secure' is not defined

In [14]:
RES_DIR = "vulns-results/dudect"
dudect_v_vulns = init_dict(vulns_names, None)
dudect_v_times = init_dict(vulns_names, "NaT")

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    stdout = load_text_output(f"{RES_DIR}/{name}-dudect.txt")
    
    dudect_v_vulns[name] = dudect_secure(stdout)
    dudect_v_times[name] = row["dudect_time"]
    
    print(f"{name}: {dudect_v_vulns[name]} in {dudect_v_times[name]}")


ec_wnaf_mul-GCC9-O2: False in 24.4988360404968
mbedtls_mpi_gcd-GCC9-O2: False in 4.263099193573
mbedtls_mpi_inv_mod-GCC9-O2: False in 17.6979668140411
mbedtls_rsa_complete-GCC9-O2: False in 2205.06592321396
bn_gcd-GCC9-O2: False in 0.697597980499268
bn_mod_exp_mont-GCC9-O2: False in 6.9636242389679
bn_mod_inverse-GCC9-O2: False in 1.68535017967224
gcry_pk_decrypt-GCC9-O2: False in 89.4567394256592
gcry_mpi_mod-GCC9-O2: False in 0.40517258644104


# Binsec/Rel

Binsec/Rel outputs statistics for each benchmark in a single csv file. However, in cases where the analysis terminates with a fatal exception (e.g. an unsupported instruction), this file isn't written, so we have to also record each benchmark's stdout.

The total running time of the analysis, as measured in our python script, is appended at the end of the file.

In [15]:
def rel_time_first(output: List[str]) -> pd.Timedelta:
    output = [line for line in output if line.startswith("[relse:result] Time:")]
    if output == []:
        # no vulnerability found
        return "NaT"
    else:
        # extracting time for the first vulnerability
        return output[0].split(" ")[2][:-2]

def rel_vuln(output: List[str]) -> List[str]:
    output = [line for line in output if line.startswith("[relse:result] Address")]
    if output == []:
        #no vulnerability found
        return []
    else:
        # extracting address of each vulnerability
        return [hex(int(line.split(" ")[2][1:-1],16)) for line in output]

def rel_secure(row):
    if row["rel1_status"] == "0":
        return True
    elif row["rel1_status"] == "7":
        return False
    else:
        return None
    
RES_DIR = f"{BENCH_RES_DIR}/binsec-rel"
rel_vulns = init_dict(binary_names, "NaN")
rel_times = init_dict(binary_names, "NaT")
rel_times_first = init_dict(binary_names, "NaT")
rel_secures = init_dict(binary_names, None)

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    stdout = load_text_output(f"{RES_DIR}/{name}-binsec.txt")
    rel_times[name] = row["rel1_time"]
    rel_secures[name] = rel_secure(row)
    rel_times_first[name] = rel_time_first(stdout)
    rel_vulns[name] = len(rel_vuln(stdout))
    
    print(f"{name}: {rel_secures[name]}, {rel_vulns[name]} in {rel_times[name]}, {rel_times_first[name]}")


aes_big_cbc-GCC9-O2: False, 20 in 3600.02001833916, 0.05442
aes_ct_cbc-GCC9-O2: True, 0 in 16.7694697380066, NaT
aes_big_gcm-GCC9-O2: False, 20 in 3600.01920437813, 0.05285
aes_ct_gcm-GCC9-O2: True, 0 in 53.3157012462616, NaT
aes_crypt_cbc-GCC9-O2: False, 20 in 3600.01991057396, 0.09674
gcm_starts-GCC9-O2: False, 20 in 3600.02212262154, 0.26983
evp_aes_cbc-GCC9-O2: None, 0 in 3600.10764098167, NaT
evp_aes_gcm-GCC9-O2: None, 0 in 3600.10626602173, NaT
aes_cbc_encrypt-GCC9-O2: False, 20 in 3600.02304506302, 0.05167
vpaes_cbc_encrypt-GCC9-O2: None, 0 in 0.058216333389282, NaT
poly1305_ctmul_chacha20_ct-GCC9-O2: True, 0 in 9.71622037887573, NaT
poly1305_ctmul_chacha20_sse2-GCC9-O2: None, 0 in 0.05308723449707, NaT
mbedtls_chachapoly-GCC9-O2: True, 0 in 18.6164882183075, NaT
evp_chacha20_poly1305-GCC9-O2: None, 0 in 3600.11765265465, NaT
chacha20_ctr32-GCC9-O2: True, 0 in 0.773261547088623, NaT
poly1305_update-GCC9-O2: True, 0 in 0.259453773498535, NaT
rsa_i31_oaep_decrypt-GCC9-O2: None, 0 

# Binsec/Rel2

In [15]:

def rel2_vuln(root: dict):
    return [hex(int(v[2:], 16)) for v in root["CT report"]["Instructions status"]["insecure"]]

def rel2_time_first(output: List[str]):
    output = [line for line in output if line.startswith("[checkct:result] Instruction")]
    if output == []:
        return "NaT"
    else:
        return output[0].split("(")[1][:-3]
    
def rel2_secure(row):
    if row["rel2_status"] == "0":
        return True
    elif row["rel2_status"] == "7":
        return False
    else:
        return None

In [17]:
RES_DIR = f"{BENCH_RES_DIR}/binsec-rel2"
rel2_vulns = init_dict(binary_names, "NaN")
rel2_times = init_dict(binary_names, "NaT")
rel2_times_first = init_dict(binary_names, "NaT")
rel2_secures = init_dict(binary_names, None)

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    stdout = load_text_output(f"{RES_DIR}/{name}-binsec-rel2.txt")
    root = toml.load(f"{RES_DIR}/{name}-binsec-rel2.toml")

    rel2_times[name] = float(row["rel2_time"])
    rel2_secures[name] = rel2_secure(row)
    rel2_times_first[name] = rel2_time_first(stdout)
    rel2_vulns[name] = len(rel2_vuln(root))
    
    print(f"{name}: {rel2_secures[name]}, {rel2_vulns[name]} in {rel2_times[name]}, {rel2_times_first[name]}")


aes_big_cbc-GCC9-O2: False, 36 in 0.102245807647705, 0.016
aes_ct_cbc-GCC9-O2: True, 0 in 0.314475536346436, NaT
aes_big_gcm-GCC9-O2: False, 36 in 0.216169595718384, 0.020
aes_ct_gcm-GCC9-O2: True, 0 in 0.484316825866699, NaT
aes_crypt_cbc-GCC9-O2: False, 68 in 0.169395923614502, 0.032
gcm_starts-GCC9-O2: False, 84 in 0.395462036132812, 0.050
evp_aes_cbc-GCC9-O2: None, 0 in 21.9165391921997, NaT
evp_aes_gcm-GCC9-O2: None, 0 in 21.1867742538452, NaT
aes_cbc_encrypt-GCC9-O2: False, 36 in 0.162734270095825, 0.019
vpaes_cbc_encrypt-GCC9-O2: True, 0 in 0.592101812362671, NaT
poly1305_ctmul_chacha20_ct-GCC9-O2: True, 0 in 0.176310300827026, NaT
poly1305_ctmul_chacha20_sse2-GCC9-O2: True, 0 in 0.111331939697266, NaT
mbedtls_chachapoly-GCC9-O2: True, 0 in 0.491197109222412, NaT
evp_chacha20_poly1305-GCC9-O2: True, 0 in 21.5532088279724, NaT
chacha20_ctr32-GCC9-O2: True, 0 in 0.094939708709717, NaT
poly1305_update-GCC9-O2: True, 0 in 0.347710132598877, NaT
rsa_i31_oaep_decrypt-GCC9-O2: False, 2

In [16]:
RES_DIR = "vulns-results/binsec-rel2"
rel2_v_vulns = init_dict(vulns_names, "NaN")
rel2_v_times = init_dict(vulns_names, "NaT")
rel2_v_secures = init_dict(binary_names, None)

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    if row["rel2_status"] == "2":
        stdout = load_text_output(f"{RES_DIR}/{name}-binsec-rel2.txt")

        rel2_v_times[name] = float(row["rel2_time"])
        rel2_v_vulns[name] = "NaN"
        rel2_v_secures[name] = None

    else:
        stdout = load_text_output(f"{RES_DIR}/{name}-binsec-rel2.txt")
        root = toml.load(f"{RES_DIR}/{name}-binsec-rel2.toml")
        
        rel2_v_times[name] = float(row["rel2_time"])
        rel2_v_vulns[name] = len(rel2_vuln(root))
        rel2_v_secures[name] = rel2_secure(row)

    print(f"{name}: {rel2_v_secures[name]},{rel2_v_vulns[name]} in {rel2_v_times[name]}")
        


ECDSA_do_sign-GCC9-O2: True,0 in 16.092079639434814
ec_wnaf_mul-GCC9-O2: False,5 in 3601.147794485092
mbedtls_rsa_complete-GCC9-O2: False,5 in 3600.0623576641083
mbedtls_mpi_gcd-GCC9-O2: False,5 in 3600.0520000457764
mbedtls_mpi_inv_mod-GCC9-O2: False,3 in 3600.0809183120728
RSA_key_generate_ex-GCC9-O2: True,0 in 0.16853761672973633
bn_gcd-GCC9-O2: False,3 in 3600.450819015503
bn_mod_exp_mont-GCC9-O2: False,1 in 3600.4430890083313
bn_mod_inverse-GCC9-O2: False,1 in 3600.7634184360504
gcry_pk_decrypt-GCC9-O2: False,2 in 3602.1406297683716
gcry_mpi_mod-GCC9-O2: None,0 in 3600.180545091629


# Microwalk

In [17]:
def microwalk_vuln(output: List[str]):
    for line in output:
        if "[analyze:cfl] Unique leaking instructions:" in line:
            return int(line.split(" ")[-1])
        
def microwalk_leakage(output: List[str], unique=True):
    pattern = r"^\s+\[L\].+:(.+) \(.+\)$"
    res = set(re.findall(pattern, "\n".join(output), flags=re.MULTILINE))
    return list(res)



In [36]:
import functools
RES_DIR = f"{BENCH_RES_DIR}/microwalk"

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    vulns = microwalk_leakage(load_text_output(f"{RES_DIR}/microwalk_{name}.txt"))
    if vulns != []:
        vulns = functools.reduce(lambda a,b: a+'\n'+b, sorted(vulns))
    print(f"> {name}:\n{vulns}\n")


> aes_big_cbc-GCC9-O2:
SubWord+15
SubWord+19
SubWord+1f
SubWord+34
br_aes_big_encrypt+126
br_aes_big_encrypt+129
br_aes_big_encrypt+137
br_aes_big_encrypt+146
br_aes_big_encrypt+159
br_aes_big_encrypt+15d
br_aes_big_encrypt+160
br_aes_big_encrypt+166
br_aes_big_encrypt+17a
br_aes_big_encrypt+182
br_aes_big_encrypt+193
br_aes_big_encrypt+1a0
br_aes_big_encrypt+1b5
br_aes_big_encrypt+1c0
br_aes_big_encrypt+1c8
br_aes_big_encrypt+1d5
br_aes_big_encrypt+1fa
br_aes_big_encrypt+1ff
br_aes_big_encrypt+20c
br_aes_big_encrypt+214
br_aes_big_encrypt+218
br_aes_big_encrypt+21e
br_aes_big_encrypt+238
br_aes_big_encrypt+251
br_aes_big_encrypt+25b
br_aes_big_encrypt+266
br_aes_big_encrypt+274
br_aes_big_encrypt+286
br_aes_big_encrypt+29f
br_aes_big_encrypt+2a8
br_aes_big_encrypt+2b2
br_aes_big_encrypt+2b7

> aes_ct_cbc-GCC9-O2:
[]

> aes_big_gcm-GCC9-O2:
SubWord+15
SubWord+19
SubWord+1f
SubWord+34
br_aes_big_encrypt+126
br_aes_big_encrypt+129
br_aes_big_encrypt+137
br_aes_big_encrypt+146
br_aes_big_

In [5]:
RES_DIR = f"{BENCH_RES_DIR}/microwalk"
microwalk_vulns = init_dict(binary_names, "NaN")
microwalk_times = init_dict(binary_names, "Nat")
microwalk_secures = init_dict(binary_names, None)

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    stdout = load_text_output(f"{RES_DIR}/{name}-microwalk.txt")
    
    microwalk_vulns[name] = microwalk_vuln(stdout)
    microwalk_times[name] = row["microwalk_time"]
    microwalk_secures[name] = None if microwalk_vulns[name] == 0 else False
    
    print(f"{name}: {microwalk_secures[name]}, {microwalk_vulns[name]} in {microwalk_times[name]}")


aes_big_cbc-GCC9-O2: False, 36 in 1.38811349868774
aes_ct_cbc-GCC9-O2: None, 0 in 1.54685258865356
aes_big_gcm-GCC9-O2: False, 36 in 1.51222658157349
aes_ct_gcm-GCC9-O2: None, 0 in 1.72842264175415
aes_crypt_cbc-GCC9-O2: False, 68 in 1.53990173339844
gcm_starts-GCC9-O2: False, 71 in 1.90111112594605
evp_aes_cbc-GCC9-O2: False, 9 in 5.34699892997742
evp_aes_gcm-GCC9-O2: False, 8 in 5.65576100349426
aes_cbc_encrypt-GCC9-O2: False, 20 in 1.37655091285706
vpaes_cbc_encrypt-GCC9-O2: None, 0 in 1.6833553314209
poly1305_ctmul_chacha20_ct-GCC9-O2: None, 0 in 1.42909789085388
poly1305_ctmul_chacha20_sse2-GCC9-O2: None, 0 in 1.34063053131104
mbedtls_chachapoly-GCC9-O2: None, 0 in 2.94519114494324
evp_chacha20_poly1305-GCC9-O2: False, 8 in 5.40686011314392
chacha20_ctr32-GCC9-O2: None, 0 in 1.36468195915222
poly1305_update-GCC9-O2: None, 0 in 1.70111203193665
rsa_i31_oaep_decrypt-GCC9-O2: None, 0 in 146.516288280487
rsa_pkcs1_v15_decrypt-GCC9-O2: False, 137 in 740.8901720047
rsa_oaep_decrypt-GCC9

In [34]:
RES_DIR = f"{VULN_RES_DIR}/microwalk"
microwalk_v_vulns = init_dict(vulns_names, "NaN")
microwalk_v_times = init_dict(vulns_names, "Nat")
microwalk_v_secures = init_dict(vulns_names, None)

stats = csv.DictReader(load_text_output(f"{RES_DIR}/benchmark_stats.csv"), delimiter=',')

for row in stats:
    name = row["name"]
    stdout = load_text_output(f"{RES_DIR}/{name}-microwalk.txt")
    
    microwalk_v_vulns[name] = microwalk_vuln(stdout)
    microwalk_v_times[name] = row["microwalk_time"]
    microwalk_v_secures[name] = None if microwalk_v_vulns[name] == 0 else False
    
    print(f"{name}: {microwalk_v_secures[name]}, {microwalk_v_vulns[name]} in {microwalk_v_times[name]}")


RSA_key_generate_ex-GCC9-O2: False, 123 in 842.018090963364
bn_gcd-GCC9-O2: False, 4 in 3.60754895210266
bn_mod_exp_mont-GCC9-O2: False, 25 in 13.5245676040649
bn_mod_inverse-GCC9-O2: False, 15 in 5.95555663108826
ECDSA_do_sign-GCC9-O2: False, 33 in 71.4390170574188
ec_wnaf_mul-GCC9-O2: False, 35 in 62.5522217750549
mbedtls_rsa_complete-GCC9-O2: False, 41 in 278.939907073975
mbedtls_mpi_gcd-GCC9-O2: False, 8 in 22.963987827301
mbedtls_mpi_inv_mod-GCC9-O2: False, 17 in 141.819532394409
gcry_pk_decrypt-GCC9-O2: False, 34 in 146.503904342651
gcry_mpi_mod-GCC9-O2: None, 0 in 1.58781003952026


# Generate latex tables

Table 2 and 3 are filled from the detection tools' outputs directly using this script. Some details must be filled manually however: core-dump initialization and early exits (table 2 and 3), and whether the right vulnerability was found (table 3). For these, a manual inspection of the output is needed.

In [30]:
import experiments as E

# names used in the paper, in the order of experiment.py
table_names = ["AES-CBC-bearssl (T)", "AES-CBC-bearssl (BS)",
               "AES-GCM-bearssl (T)", "AES-GCM-bearssl (BS)",
               "AES-CBC-mbedtls (T)", "AES-GCM-mbedtls (T)",
               "AES-CBC-openssl (EVP)", "AES-GCM-openssl (EVP)",
               "AES-CBC-openssl (T)", "AES-CBC-openssl (VP)",
               "PolyChacha-bearssl (CT)", "PolyChacha-bearssl (SSE2)",
               "PolyChacha-mbedtls", "PolyChacha-openssl (EVP)",
               "Chacha20-openssl", "Poly1305-openssl", 
               "RSA-bearssl (OAEP)", "RSA-mbedtls (PKCS)",
               "RSA-mbedtls (OAEP)", "RSA-openssl (PKCS)",
               "RSA-openssl (OAEP)",
               "ECDSA-bearssl", "ECDSA-mbedtls", 
               "ECDSA-openssl", 
               "EdDSA-openssl"]

for i, b in enumerate(binary_names):
    # lambda to convert values into the strings used in the table
    si = lambda x: "\\crash" if x == "NaN" else x
    sb = lambda x: "\\true" if x is True else "\\false" if x is False else "\\unknown"
    st = lambda x: "--" if x == "NaT" else "\\timeout" if float(x) >= 3599 else str(np.format_float_positional(float(x), precision=2, fractional=True))
    row = (table_names[i] + " & " 
          f"{si(rel_vulns[b])} & {sb(rel_secures[b])} & {st(rel_times_first[b])} & {st(rel_times[b])} & "
          f"{si(rel2_vulns[b])} & {sb(rel2_secures[b])} & {st(rel2_times_first[b])} & {st(rel2_times[b])} & "
          f"{si(abacus_vulns[b])} & {sb(abacus_secures[b])} & {st(abacus_times[b])} & "
          f"{si(ctgrind_vulns[b])} & {sb(ctgrind_secures[b])} & {st(ctgrind_times[b])} & "
          f"{si(microwalk_vulns[b])} & {sb(microwalk_secures[b])} & {st(microwalk_times[b])} & "
          f"{sb(dudect_secures[b])} & {st(dudect_times[b])} \\\\")
    print(row)

AES-CBC-bearssl (T) & 20 & \false & 0.05 & \timeout & 36 & \false & 0.02 & 0.10 & 36 & \false & 3.65 & 36 & \false & 0.16 & 36 & \false & 1.39 & \false & 100.51 \\
AES-CBC-bearssl (BS) & 0 & \true & -- & 16.77 & 0 & \true & -- & 0.31 & 0 & \unknown & 10.69 & 0 & \unknown & 0.17 & 0 & \unknown & 1.55 & \unknown & \timeout \\
AES-GCM-bearssl (T) & 20 & \false & 0.05 & \timeout & 36 & \false & 0.02 & 0.22 & 36 & \false & 6.23 & 36 & \false & 0.18 & 36 & \false & 1.51 & \false & 8.69 \\
AES-GCM-bearssl (BS) & 0 & \true & -- & 53.32 & 0 & \true & -- & 0.48 & 0 & \unknown & 14.79 & 0 & \unknown & 0.17 & 0 & \unknown & 1.73 & \unknown & \timeout \\
AES-CBC-mbedtls (T) & 20 & \false & 0.1 & \timeout & 68 & \false & 0.03 & 0.17 & 68 & \false & 5.73 & 68 & \false & 0.19 & 68 & \false & 1.54 & \false & 0.63 \\
AES-GCM-mbedtls (T) & 20 & \false & 0.27 & \timeout & 84 & \false & 0.05 & 0.4 & 84 & \false & 261.76 & 76 & \false & 0.19 & 71 & \false & 1.90 & \false & 0.89 \\
AES-CBC-openssl (EVP) & 0 

In [33]:
import experiments as E

# names used in the paper, in the order of experiment.py
table_names = ["\\textbf{P256 sign (OpenSSL)}", "\\ wNAF mul. (OpenSSL)",
              "\\textbf{RSA valid. (MbedTLS)}", "\\ GCD (MbedTLS)", "\\ Mod. inv. (MbedTLS)",
              "\\textbf{RSA keygen (OpenSSL)}", "\\ GCD (OpenSSL)", "\\ Mod. exp. (OpenSSL)", "\\ Mod. inv. (OpenSSL)",
              "\\textbf{ECDH decrypt. (Libgcrypt)}", "\\ Mod. (Libgcrypt)"]

for i, b in enumerate(vulns_names):
    # lambda to convert values into the strings used in the table
    si = lambda x: "NA" if x == "NaN" else x
    sb = lambda x: "\\true" if x is True else ("\\false" if x is False else "\\unknown")
    st = lambda x: "--" if x == "NaT" else "\\timeout" if float(x) >= 3599 else str(np.format_float_positional(float(x), precision=2, fractional=True))
    row = (table_names[i] + " & " 
          f" & {si(rel2_v_vulns[b])} & {sb(rel2_v_secures[b])} & {st(rel2_v_times[b])} & "
          f" & {si(abacus_v_vulns[b])} & {sb(abacus_v_secures[b])} & {st(abacus_v_times[b])} & "
          f" & {si(ctgrind_v_vulns[b])} & {sb(ctgrind_v_secures[b])} & {st(ctgrind_v_times[b])} & "
          f" & {si(microwalk_v_vulns[b])} & {sb(microwalk_v_secures[b])} & {st(microwalk_v_times[b])} & "
          f"{sb(dudect_v_vulns[b])} & {st(dudect_v_times[b])} \\\\")
    print(row)

\textbf{P256 sign (OpenSSL)} &  & 0 & \true & 16.09 &  & NA & \unknown & 5.69 &  & 0 & \unknown & 0.75 &  & 33 & \false & 71.44 & \unknown & -- \\
\ wNAF mul. (OpenSSL) &  & 5 & \false & \timeout &  & NA & \unknown & \timeout &  & 222 & \false & 0.54 &  & 35 & \false & 62.55 & \false & 24.5 \\
\textbf{RSA valid. (MbedTLS)} &  & 5 & \false & \timeout &  & 0 & \unknown & 490.01 &  & 31 & \false & 0.40 &  & 41 & \false & 278.94 & \false & 2205.07 \\
\ GCD (MbedTLS) &  & 5 & \false & \timeout &  & 0 & \unknown & 37.74 &  & 14 & \false & 0.21 &  & 8 & \false & 22.96 & \false & 4.26 \\
\ Mod. inv. (MbedTLS) &  & 3 & \false & \timeout &  & 0 & \unknown & 242.1 &  & 19 & \false & 0.24 &  & 17 & \false & 141.82 & \false & 17.7 \\
\textbf{RSA keygen (OpenSSL)} &  & 0 & \true & 0.17 &  & NA & \unknown & 8.66 &  & 0 & \unknown & 6.36 &  & 123 & \false & 842.02 & \unknown & -- \\
\ GCD (OpenSSL) &  & 3 & \false & \timeout &  & NA & \unknown & \timeout &  & 53 & \false & 0.19 &  & 4 & \false & 3.61 

## Vulnerability details


In [35]:
name = "gcm_starts-GCC9-O2"

# tools output for this binary
rel1_out = load_text_output(f"{BENCH_RES_DIR}/binsec-rel/{name}-binsec.txt")
rel2_out = toml.load(f"{BENCH_RES_DIR}/binsec-rel2/{name}-binsec-rel2.toml")
ctgrind_out = load_ctgrind_output(f"{BENCH_RES_DIR}/ctgrind/{name}-ctgrind.txt")
abacus_out = load_text_output(f"{BENCH_RES_DIR}/abacus/{name}-abacus.txt")

vuln_list = list(set(rel2_vuln(rel2_out)) | set(rel_vuln(rel1_out)) | set(abacus_vuln(abacus_out)) | set(ctgrind_vuln(ctgrind_out)))

rel2_list = np.array([(v in rel2_vuln(rel2_out)) for v in vuln_list])
rel_list = np.array([(v in rel_vuln(rel1_out)) for v in vuln_list])
abacus_list = np.array([(v in abacus_vuln(abacus_out)) for v in vuln_list])
ctgrind_list = np.array([(v in ctgrind_vuln(ctgrind_out)) for v in vuln_list])

print(len(rel_list[rel_list == True]))
print(len(rel2_list[rel2_list == True]))
print(len(abacus_list[abacus_list == True]))
print(len(ctgrind_list[ctgrind_list == True]))

df_vulns = pd.DataFrame(np.transpose([
    [(v in rel_vuln(rel1_out)) for v in vuln_list],
    [(v in rel2_vuln(rel2_out)) for v in vuln_list],
    [(v in abacus_vuln(abacus_out)) for v in vuln_list],
    [(v in ctgrind_vuln(ctgrind_out)) for v in vuln_list]]),
    index=vuln_list, columns=["Rel1", "Rel2", "Abacus", "ctgrind"])

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_vulns)


20
84
84
76
            Rel1  Rel2  Abacus  ctgrind
0x8060d6d   True  True    True     True
0x804a78a  False  True    True     True
0x804a83e  False  True    True    False
0x804a83c  False  True    True     True
0x804a86d  False  True    True    False
0x8060e30  False  True    True     True
0x804a689  False  True    True    False
0x806023a   True  True    True     True
0x806111e  False  True    True     True
0x8060ddd  False  True    True     True
0x8061030  False  True    True     True
0x8060273   True  True    True     True
0x8061013  False  True    True     True
0x8061001  False  True    True     True
0x8060ee8  False  True    True     True
0x8060dae   True  True    True     True
0x8060d1a   True  True    True     True
0x8060d49   True  True    True     True
0x8060ed6  False  True    True     True
0x804a7fa  False  True    True    False
0x8060e42  False  True    True     True
0x80610cb  False  True    True     True
0x804a66e  False  True    True     True
0x8060f37  False  True    Tr

In [330]:
name_pkcs = "RSA_private_decrypt_pkcs1_v15-GCC9-O2"
name_oaep = "RSA_private_decrypt_oaep-GCC9-O2"
RUN_NUMBER = 100

vulns_lists_pkcs = []
vulns_lists_oaep = []

for i in range(1,RUN_NUMBER):
    run = load_ctgrind_output(f"ctgrind-no-blinding/ctgrind{i}/{name_pkcs}-ctgrind.txt")
    vulns_lists_pkcs.append(ctgrind_vuln(run))
    run = load_ctgrind_output(f"ctgrind-no-blinding/ctgrind{i}/{name_oaep}-ctgrind.txt")
    vulns_lists_oaep.append(ctgrind_vuln(run))


In [331]:
from functools import reduce

mean_pkcs = np.mean([len(l) for l in vulns_lists_pkcs])
mean_oaep = np.mean([len(l) for l in vulns_lists_oaep])

print(name_pkcs)

union_pkcs = reduce(np.union1d, vulns_lists_pkcs)
inter_pkcs = reduce(np.intersect1d, vulns_lists_pkcs)
print(f"total: {len(union_pkcs)}, common: {len(inter_pkcs)}, mean: {mean_pkcs}")
print(f"difference: {np.setdiff1d(union_pkcs, inter_pkcs)}")

print(name_pkcs)

union_oaep = reduce(np.union1d, vulns_lists_oaep)
inter_oaep = reduce(np.intersect1d, vulns_lists_oaep)
print(f"total: {len(union_oaep)}, common: {len(inter_oaep)}, mean: {mean_oaep}")
print(f"difference: {np.setdiff1d(union_oaep, inter_oaep)}")

RSA_private_decrypt_pkcs1_v15-GCC9-O2
total: 393, common: 393, mean: 393.0
difference: []
RSA_private_decrypt_pkcs1_v15-GCC9-O2
total: 542, common: 540, mean: 541.5050505050505
difference: ['0x804e97b' '0x804e97f']
